Uses pyDBML. [https://pypi.org/project/pydbml/](https:\pypi.org\project\pydbml\)  Of course there's a module for it....  
https://github.com/Vanderhoof/PyDBML/blob/master/docs/classes.md  
Imports a DBML file as output by the dbdiagram.io website to the Phyiscal Structure, Physical Attribute, and Foreign Key hubs.  
Expected format:

Table \[\<schema\>.\]\<tablename\> {

  \<fieldname\> \<datatype\> \['\[pk\]'\]

  \<fieldname\> \<datatype\>

  ...

}

Ref: \[\<schema\>.\]\<tablename\>.\<fieldname\> \> \[\<schema\>.\]\<tablename\>.\<fieldname\> \['\[note: "\<enforcementtype\>"\]'\]

...

In [1]:
script_depth = '../../'

import sqlite3
import pandas as pd
import numpy as np
from pydbml import PyDBML
import os
from datetime import datetime
import sys
sys.path.append(script_depth)
from user_packages import hashing

In [2]:
# Set variables
target_model_name = 'contain.pig.tall' # from bv_Model
source_file_name = 'Scheduling_ERD_DBML_20240902.txt'
source_file_path = script_depth + 'imports/physical/DBML/'

record_source = 'DBML_txt'
SystemKeyPhrase = 'iPropertyCloud' # this should come from the System Hub, for linking structure to system
SERVER_NAME = 'ipcdevprg.crm11.dynamics.com' # the server you harvested from
DATABASE_NAME = 'ipcdevprg' # the database you harvested from
DEFAULT_SCHEMA_NAME = 'dbo' # the default schema for tables seen in the DBML. This is overridden if there is an actual schema defined, per table/link.

In [14]:
# Load DBML file
with open(os.path.join(source_file_path, source_file_name), 'r') as f:
  data = PyDBML(f)

# load db
conn = sqlite3.connect(script_depth + 'full_metadata.db')

In [9]:
print(data.tables[0].schema)

public


In [15]:
# Extract table information
table_dict_list = []
for table in data.tables:
  table_dict_list.append({
    'TABLE_NAME': table.name,
    'SCHEMA_NAME': DEFAULT_SCHEMA_NAME if(table.schema == 'public') else table.schema
  })

df_tables = pd.DataFrame(table_dict_list)

#df_tables

In [39]:
# Extract field information
# note that the data type isn't broken down in any way, so there would need to be custom parsing for 'decimal(15,2)'
column_dict_list = []
for table in data.tables:
  for column in table.columns:
    column_dict_list.append({
      'TABLE_NAME': table.name,
      'SCHEMA_NAME': DEFAULT_SCHEMA_NAME if(table.schema == 'public') else table.schema,
      'COLUMN_NAME': column.name,
      'DATA_TYPE': column.type # TODO: this can be an Enum object, but this isn't handled right now
    })

df_columns = pd.DataFrame(column_dict_list)

#df_columns

In [28]:
# Extract REF information
# TODO: It's good that the DBML library handles multi-column keys. It's bad that I don't right now.
ref_dict_list = []
for ref in data.refs:
  ref_dict_list.append({
    'SCHEMA_NAME_CHILD': DEFAULT_SCHEMA_NAME if(ref.table1.schema == 'public') else ref.table1.schema,
    'TABLE_NAME_CHILD': ref.table1.name,
    'COLUMN_NAME_CHILD': ref.col1[0].name,
    'SCHEMA_NAME_PARENT': DEFAULT_SCHEMA_NAME if(ref.table2.schema == 'public') else ref.table2.schema,
    'TABLE_NAME_PARENT': ref.table2.name,
    'COLUMN_NAME_PARENT': ref.col2[0].name,
    'ORDINAL_POSITION': 1,
    'CONSTRAINT_NAME': ref.name if(ref.name != None) else 'fk_' + ref.table1.name + '_' + ref.col1[0].name # TODO: This needs a more reliable way of creating a name
  })

df_refs = pd.DataFrame(ref_dict_list)

#df_refs

In [30]:
# Physical Structure processing
# add extra columns
df = df_tables

# columns with variable data
df['RecordSource'] = record_source
df['LoadDate'] = datetime.now()
df['ModelKeyPhrase'] = target_model_name
df['SystemKeyPhrase'] = SystemKeyPhrase
df['SERVER_NAME'] = SERVER_NAME
df['DATABASE_NAME'] = DATABASE_NAME

# derive keyphrase columns
df['PhysicalStructureKeyPhrase'] = df.apply(
  lambda row: 
    row['SERVER_NAME']
    + '.' + row['DATABASE_NAME']
    + '.' + row['SCHEMA_NAME']
    + '.' + row['TABLE_NAME'] # This corresponds to the Logical Name
  , axis=1
)
df['ModelPhysicalStructureKeyPhrase'] = df.apply(  lambda row: row['ModelKeyPhrase'] + ':' + row['PhysicalStructureKeyPhrase'], axis=1)
df['SystemPhysicalStructureKeyPhrase'] = df.apply(  lambda row: row['SystemKeyPhrase'] + ':' + row['PhysicalStructureKeyPhrase'], axis=1)

# hash the keyphrases
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelHashKey'
  , columns = ['ModelKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'SystemHashKey'
  , columns = ['SystemKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'PhysicalStructureHashKey'
  , columns = ['PhysicalStructureKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelPhysicalStructureHashKey'
  , columns = ['ModelPhysicalStructureKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'SystemPhysicalStructureHashKey'
  , columns = ['SystemPhysicalStructureKeyPhrase']
)


# hash the payload
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = [
      'SERVER_NAME'
    , 'DATABASE_NAME'
    , 'SCHEMA_NAME'
    , 'TABLE_NAME'
  ]
)

#df


In [31]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_DBMLToPhysicalStructure")
conn.commit()
df.to_sql('stg_Py_DBMLToPhysicalStructure', conn, if_exists='append', index=False)

12

In [32]:
# Write to the Hub (PhysicalStructure)
sql_query = """
INSERT INTO rv_h_PhysicalStructure
(
    PhysicalStructureHashKey
  , LoadDate
  , RecordSource
  , PhysicalStructureKeyPhrase
)
SELECT DISTINCT
    PhysicalStructureHashKey
  , LoadDate
  , RecordSource
  , PhysicalStructureKeyPhrase
FROM
  stg_Py_DBMLToPhysicalStructure
WHERE
  PhysicalStructureHashKey NOT IN (SELECT PhysicalStructureHashKey FROM rv_h_PhysicalStructure)
""";
conn.execute(sql_query)
conn.commit()

In [34]:
# Write to the Satellite (PhysicalStructure_DBMLtxt)
sql_query = """
INSERT INTO rv_s_PhysicalStructure_DBMLtxt
(
    PhysicalStructureHashKey
  , LoadDate
  , RecordSource
  , HashDiff

  , "SERVER_NAME"
  , "DATABASE_NAME"
  , "SCHEMA_NAME"
  , "TABLE_NAME"

)
SELECT DISTINCT
    stg.PhysicalStructureHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  
  , stg."SERVER_NAME"
  , stg."DATABASE_NAME"
  , stg."SCHEMA_NAME"
  , stg."TABLE_NAME"

FROM
  stg_Py_DBMLToPhysicalStructure AS stg
  LEFT OUTER JOIN rv_s_PhysicalStructure_DBMLtxt AS sat ON (
    stg.PhysicalStructureHashKey = sat.PhysicalStructureHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_PhysicalStructure_DBMLtxt AS z
      WHERE z.PhysicalStructureHashKey = sat.PhysicalStructureHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()

In [35]:
# Write to the Link (Model - PhysicalStructure)
sql_query = """
INSERT INTO rv_l_ModelPhysicalStructure
(
  ModelPhysicalStructureHashKey
  , LoadDate
  , RecordSource
  , ModelHashKey
  , PhysicalStructureHashKey
)
SELECT
  stg.ModelPhysicalStructureHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.ModelHashKey
  , stg.PhysicalStructureHashKey
FROM
  stg_Py_DBMLToPhysicalStructure stg
WHERE
  stg.ModelPhysicalStructureHashKey IS NOT NULL
  AND ModelPhysicalStructureHashKey NOT IN (
    SELECT ModelPhysicalStructureHashKey
    FROM rv_l_ModelPhysicalStructure
  )
""";
conn.execute(sql_query)
conn.commit()

In [36]:
# Write to the Link (System - PhysicalStructure)
sql_query = """
INSERT INTO rv_l_SystemPhysicalStructure
(
  SystemPhysicalStructureHashKey
  , LoadDate
  , RecordSource
  , SystemHashKey
  , PhysicalStructureHashKey
)
SELECT
  stg.SystemPhysicalStructureHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.SystemHashKey
  , stg.PhysicalStructureHashKey
FROM
  stg_Py_DBMLToPhysicalStructure stg
WHERE
  stg.SystemPhysicalStructureHashKey IS NOT NULL
  AND SystemPhysicalStructureHashKey NOT IN (
    SELECT SystemPhysicalStructureHashKey
    FROM rv_l_SystemPhysicalStructure
  )
""";
conn.execute(sql_query)
conn.commit()

In [40]:
# Processing the attributes
df = df_columns

# add extra columns

# columns with variable data
df['RecordSource'] = record_source
df['LoadDate'] = datetime.now()
df['ModelKeyPhrase'] = target_model_name
df['SERVER_NAME'] = SERVER_NAME
df['DATABASE_NAME'] = DATABASE_NAME

# derive keyphrase columns
df['PhysicalStructureKeyPhrase'] = df.apply(
  lambda row: 
    row['SERVER_NAME']
    + '.' + row['DATABASE_NAME']
    + '.' + row['SCHEMA_NAME']
    + '.' + row['TABLE_NAME']
  , axis=1
)
df['PhysicalAttributeKeyPhrase'] = df.apply(
  lambda row: 
    row['SERVER_NAME']
    + '.' + row['DATABASE_NAME']
    + '.' + row['SCHEMA_NAME']
    + '.' + row['TABLE_NAME']
    + '.' + row['COLUMN_NAME']
  , axis=1
)
df['PhysicalStructurePhysicalAttributeKeyPhrase'] = df.apply(  lambda row: row['PhysicalStructureKeyPhrase'] + ':' + row['PhysicalAttributeKeyPhrase'], axis=1)

# hash the keyphrases
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'PhysicalStructureHashKey'
  , columns = ['PhysicalStructureKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'PhysicalAttributeHashKey'
  , columns = ['PhysicalAttributeKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'PhysicalStructurePhysicalAttributeHashKey'
  , columns = ['PhysicalStructurePhysicalAttributeKeyPhrase']
)

# hash the payload
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = [
      'SERVER_NAME'
    , 'DATABASE_NAME'
    , 'SCHEMA_NAME'
    , 'TABLE_NAME'
    , 'COLUMN_NAME'
    , 'DATA_TYPE'
  ]
)

#df

In [41]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_DBMLToPhysicalAttribute")
conn.commit()
df.to_sql('stg_Py_DBMLToPhysicalAttribute', conn, if_exists='append', index=False)

40

In [42]:
# Write to the Hub (PhysicalAttribute)
sql_query = """
INSERT INTO rv_h_PhysicalAttribute
(
    PhysicalAttributeHashKey
  , LoadDate
  , RecordSource
  , PhysicalAttributeKeyPhrase
)
SELECT DISTINCT
    PhysicalAttributeHashKey
  , LoadDate
  , RecordSource
  , PhysicalAttributeKeyPhrase
FROM
  stg_Py_DBMLToPhysicalAttribute
WHERE
  PhysicalAttributeHashKey NOT IN (SELECT PhysicalAttributeHashKey FROM rv_h_PhysicalAttribute)
""";
conn.execute(sql_query)
conn.commit()

In [43]:
# Write to the Satellite (PhysicalAttribute_DBMLtxt)
sql_query = """
INSERT INTO rv_s_PhysicalAttribute_DBMLtxt
(
    PhysicalAttributeHashKey
  , LoadDate
  , RecordSource
  , HashDiff

  , "SERVER_NAME"
  , "DATABASE_NAME"
  , "SCHEMA_NAME"
  , "TABLE_NAME"
  , "COLUMN_NAME"
  , "DATA_TYPE"

)
SELECT DISTINCT
    stg.PhysicalAttributeHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  
  , stg."SERVER_NAME"
  , stg."DATABASE_NAME"
  , stg."SCHEMA_NAME"
  , stg."TABLE_NAME"
  , stg."COLUMN_NAME"
  , stg."DATA_TYPE"

FROM
  stg_Py_DBMLToPhysicalAttribute AS stg
  LEFT OUTER JOIN rv_s_PhysicalAttribute_DBMLtxt AS sat ON (
    stg.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_PhysicalAttribute_DBMLtxt AS z
      WHERE z.PhysicalAttributeHashKey = sat.PhysicalAttributeHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()

In [44]:
# Write to the Link (PhysicalStructure - PhysicalAttribute)
sql_query = """
INSERT INTO rv_l_PhysicalStructurePhysicalAttribute
(
  PhysicalStructurePhysicalAttributeHashKey
  , LoadDate
  , RecordSource
  , PhysicalStructureHashKey
  , PhysicalAttributeHashKey
)
SELECT
  stg.PhysicalStructurePhysicalAttributeHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.PhysicalStructureHashKey
  , stg.PhysicalAttributeHashKey
FROM
  stg_Py_DBMLToPhysicalAttribute stg
WHERE
  stg.PhysicalStructurePhysicalAttributeHashKey IS NOT NULL
  AND PhysicalStructurePhysicalAttributeHashKey NOT IN (
    SELECT PhysicalStructurePhysicalAttributeHashKey
    FROM rv_l_PhysicalStructurePhysicalAttribute
  )
""";
conn.execute(sql_query)
conn.commit()

In [45]:
# Processing the Foreign Keys
df = df_refs

# add extra columns

# columns with variable data
df['RecordSource'] = record_source
df['LoadDate'] = datetime.now()
df['SERVER_NAME_CHILD'] = SERVER_NAME
df['DATABASE_NAME_CHILD'] = DATABASE_NAME
df['SERVER_NAME_PARENT'] = SERVER_NAME
df['DATABASE_NAME_PARENT'] = DATABASE_NAME
df['CONSTRAINT_ENFORCEMENT'] = 'LOGICAL' # TODO: This absolutely isn't true, but I don't know how to work this in right now

# derive keyphrase columns
df['PhysicalAttribute_childKeyPhrase'] = df.apply(
  lambda row: 
    row['SERVER_NAME_CHILD']
    + '.' + row['DATABASE_NAME_CHILD']
    + '.' + row['SCHEMA_NAME_CHILD']
    + '.' + row['TABLE_NAME_CHILD']
    + '.' + row['COLUMN_NAME_CHILD']
  , axis=1
)
df['PhysicalAttribute_parentKeyPhrase'] = df.apply(
  lambda row: 
    row['SERVER_NAME_PARENT']
    + '.' + row['DATABASE_NAME_PARENT']
    + '.' + row['SCHEMA_NAME_PARENT']
    + '.' + row['TABLE_NAME_PARENT']
    + '.' + row['COLUMN_NAME_PARENT']
  , axis=1
)


# hash the keyphrases
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'PhysicalAttribute_childHashKey'
  , columns = ['PhysicalAttribute_childKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'PhysicalAttribute_parentHashKey'
  , columns = ['PhysicalAttribute_parentKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'PhysicalForeignKeyHashKey'
  , columns = ['CONSTRAINT_NAME']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'PhysicalForeignKeyPhysicalAttributePhysicalAttributeHashKey'
  , columns = [
    'CONSTRAINT_NAME'
    , 'PhysicalAttribute_childKeyPhrase'
    , 'PhysicalAttribute_parentKeyPhrase'
    , 'ORDINAL_POSITION'
  ]
)


# hash the payload
# As this is essentially an effectivity satellite, the only field that gets hashed is the enforcement.
# Start/end dates are dealt with in the queries.
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = [
      'CONSTRAINT_ENFORCEMENT'
  ]
)

#df

In [46]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_ForeignKeyCsvToPhysicalForeignKey")
conn.commit()
df.to_sql('stg_Py_ForeignKeyCsvToPhysicalForeignKey', conn, if_exists='append', index=False)

28

In [47]:
# Write to the Hub
sql_query = """
INSERT INTO rv_h_PhysicalForeignKey
(
    PhysicalForeignKeyHashKey
  , LoadDate
  , RecordSource
  , PhysicalForeignKeyName
)
SELECT DISTINCT
    PhysicalForeignKeyHashKey
  , LoadDate
  , RecordSource
  , CONSTRAINT_NAME
FROM
  stg_Py_ForeignKeyCsvToPhysicalForeignKey
WHERE
  PhysicalForeignKeyHashKey NOT IN (SELECT PhysicalForeignKeyHashKey FROM rv_h_PhysicalForeignKey)
""";
conn.execute(sql_query)
conn.commit()

In [48]:
# Write to the Satellite 
# This only starts a new effectivity row, does not end
sql_query = """
INSERT INTO rv_s_PhysicalForeignKey
(
    PhysicalForeignKeyHashKey
  , LoadDate
  , RecordSource
  , HashDiff

  , Enforcement
  , EffectiveStart

)
SELECT DISTINCT
    stg.PhysicalForeignKeyHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  
  , stg.CONSTRAINT_ENFORCEMENT
  , datetime()

FROM
  stg_Py_ForeignKeyCsvToPhysicalForeignKey AS stg
  LEFT OUTER JOIN rv_s_PhysicalForeignKey AS sat ON (
    stg.PhysicalForeignKeyHashKey = sat.PhysicalForeignKeyHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_PhysicalForeignKey AS z
      WHERE z.PhysicalForeignKeyHashKey = sat.PhysicalForeignKeyHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()

In [49]:
# Write to the Link
sql_query = """
INSERT INTO rv_l_PhysicalForeignKeyPhysicalAttributePhysicalAttribute
(
  PhysicalForeignKeyPhysicalAttributePhysicalAttributeHashKey
  , LoadDate
  , RecordSource
  , PhysicalForeignKeyHashKey
  , PhysicalAttribute_childHashKey
  , PhysicalAttribute_parentHashKey
  , OrdinalPosition 
)
SELECT
  stg.PhysicalForeignKeyPhysicalAttributePhysicalAttributeHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.PhysicalForeignKeyHashKey
  , stg.PhysicalAttribute_childHashKey
  , stg.PhysicalAttribute_parentHashKey
  , stg.ORDINAL_POSITION 
FROM
  stg_Py_ForeignKeyCsvToPhysicalForeignKey stg
WHERE
  stg.PhysicalForeignKeyPhysicalAttributePhysicalAttributeHashKey IS NOT NULL
  AND PhysicalForeignKeyPhysicalAttributePhysicalAttributeHashKey NOT IN (
    SELECT PhysicalForeignKeyPhysicalAttributePhysicalAttributeHashKey
    FROM rv_l_PhysicalForeignKeyPhysicalAttributePhysicalAttribute
  )
""";
conn.execute(sql_query)
conn.commit()